# SSNE Miniproject 5
### 318703 Tomasz Owienko
### 318718 Anna Schäfer
### Grupa piątek

In [12]:
import torch
import torchvision.transforms as transforms
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS, EVAL_DATALOADERS
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import pytorch_lightning as pl
import pickle
import numpy as np
import torch.nn as nn

In [13]:
RANDOM_SEED = 123
pl.seed_everything(RANDOM_SEED)

Seed set to 123


123

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.set_float32_matmul_precision('medium')

In [15]:
train_pkl = pickle.load(open("data/train.pkl", "rb"))

In [33]:
np.random.shuffle(train_pkl)

In [34]:
train_pkl

[(array([ 32.,  32., 146.,  34.,   2.,  65., 146.,  32., 144., 144., 144.,
         146.,  88.,  33.,  36.,  20.,  44., 180.,  33.,  93., 156.,   6.,
         146.,  25.,  47.,   7., 127.,  79., 158., 116.,  58., 157.,  13.,
         127., 121.,  30.,  40.,  44.,  44., 121.,  60.,  12., 159.,  92.,
          13.,  47., 156.,  78.,  78.,  12.,  69.,  45.,  13., 159.,  28.,
          41.,  12.,  71.,  12., 152.,  12.,  47.,  47., 157.,  92., 121.,
          88.,  44.,  78.,  20.,  13., 172., 127.,  60., 127.,  92.,  92.,
          44.,  60., 125.,  28.,  13.,  92., 158., 158.,  28.,  15., 140.,
          46., 172., 172.,  13., 124., 140.,  46.,  93., 190., 103.,  60.,
         140.,   6.,  79.,  45., 172., 158., 153.,  93., 140.,  46.,  39.,
         124., 120.,  60.,   9.,  15., 140.,   6.,  15., 140., 125.,  12.,
         140.,  15.,  46.,  28., 140., 127.,  46.,  15.,  15.,  15.,  15.,
          15., 127.,  28.,  92.,  28., 172., 172.,  15.,  15., 127.,  60.,
         190.,  12.,  12.

In [35]:
classes = {0: 'bach', 1: 'beethoven', 2: 'debussy', 3: 'scarlatti', 4: 'victoria'}

In [39]:
from torch.utils.data import Dataset

class VariableLenDataset(Dataset):
    def __init__(self, data):
        self.data = data      

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data, target = self.data[idx]
        return in_data, target

In [37]:
train_indices = int(len(train_pkl) * 0.7)
train_indices

2057

In [40]:
train_indices = int(len(train_pkl) * 0.7)
train_set = VariableLenDataset(train_pkl[:train_indices])
test_set = VariableLenDataset(train_pkl[train_indices:])

In [41]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

pad = 0

def pad_collate(batch, pad_value=pad):
    xx, yy = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens

In [42]:
train_loader = DataLoader(train_set, batch_size=50, shuffle=True, collate_fn=pad_collate)
test_loader = DataLoader(test_set, batch_size=50, shuffle=False, drop_last=False, collate_fn=pad_collate)

In [43]:
print(len(train_set[0][0]))
print(len(train_set[1][0]))

492
68


In [44]:
next(iter(train_loader))

TypeError: object of type 'int' has no len()

In [45]:
class LSTM_Seq_Regressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.proj_size = out_size
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, proj_size = out_size)
        
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.proj_size)
        state = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, state
    
    def forward(self, x, hidden):
        # x = torch.transpose(x, 0, 1)
        all_outputs, hidden = self.lstm(x, hidden)
        # all_outputs = torch.transpose(all_outputs, 0, 1)
        return all_outputs, hidden
    
model = LSTM_Seq_Regressor(1, 200, 1, 1).to(device)
model

LSTM_Seq_Regressor(
  (lstm): LSTM(1, 200, proj_size=1)
)

In [46]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fun = nn.MSELoss()

# Training loop
for epoch in range(101):
    for x, targets, x_len, target_len in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device) 
        
        x = torch.transpose(x, 0, 1)
        preds, _ = model(x, (hidden, state))
        preds = torch.transpose(preds, 0, 1)
        
#         x_packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
#         preds_packed, _ = model(x_packed, (hidden, state))
#         preds, pred_len = pad_packed_sequence(preds_packed, batch_first=True, padding_value=pad)
        
        preds = preds.squeeze(2)
        optimizer.zero_grad()
        mask = targets != pad
        loss = loss_fun(preds[mask], targets[mask])
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}")

TypeError: object of type 'int' has no len()

In [ ]:
with torch.no_grad():
    for x, targets, x_len, target_len in test_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.shape[0])
        hidden, state = hidden.to(device), state.to(device)

        x = torch.transpose(x, 0, 1)        
        preds, (last_hidden, last_state) = model(x, (hidden, state))
        preds = torch.transpose(preds, 0, 1)
        
#         x_packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
#         preds_packed, (last_hidden, last_state) = model(x_packed, (hidden, state))
#         preds, pred_len = pad_packed_sequence(preds_packed, batch_first=True, padding_value=pad)
        
        preds = preds.squeeze(2)
        mask_tgt = targets != pad
        print(last_hidden)
#         print(targets)
#         print(preds)
#         print(torch.abs(preds[mask_tgt] - targets[mask_tgt]).mean())
#         print()